#Note#
Test the script: Once the script has been written, it should be thoroughly tested to ensure that it is functioning correctly. This may include running the script with a variety of different input parameters to verify that it is downloading the correct data.


#Section 1 <font color='red'>**Downloader**</font> #



The following document outlines instructions for implementing a data downloading script to automatically download CFS data for a specific range of dates and variables.

Instructions:

1. Initial set up: Ensure that you have access to the necessary data for following evaluation work. This may include access to the CFS data source, as well as any necessary software or tools required for downloading.

2. Define the input parameters: The command line should accept input parameters for the initial date, end of the simulation, and variables of interest. This will allow the script to automatically download the relevant data for the specified time range and variables.

3. The script should then download the requested data and save it to a specified location on the local machine.


By following these instructions, use should be able to implement a data downloading script that can automatically download CFS data for a specific range of dates and variables. This will allow you to efficiently retrieve the necessary data for your research or analysis, without having to manually download it from the data source.


##Naming convention##
We will follow the same naming convention. Say example as following


```
"CWRF_AT2M_2010-09-01-06_2011-10-01-12_E01_2010-06-01.nc"
```
This name can be generated by the following code template:

```
source,vname = "CWRF","AT2M"

f"{source}_{vname}_{time_beg}_{time_end}_E{exp}_{time_IC}.nc"
```




In [ ]:
from subprocess import check_call
import datetime 
# all capital is a constant value used in this project
# we might want to put them in a seperate folder, whereas it will become slow 

data_dir = "/ocean/projects/ees210011p/shared/CWRF_CFS_seasonal_forecasting/DATA/"

TIMEFMT = '%Y-%m-%d-%H'
source ="CWRF" # note if this is a raw CFS (just downloaded), we call CFSraw
# whereas the CFS has been projected we call CFS

vname = "AT2M"
time_beg = datetime.datetime(2010, 9, 1, 6).strftime(TIMEFMT)
time_end = datetime.datetime(2011, 10, 1, 12).strftime(TIMEFMT)
time_IC = datetime.datetime(2010, 6, 1, 6).strftime(TIMEFMT)
exp = "01"    # experiment ID, not limited to numbere, can be 01a for example
filefmt = "nc" # can be either nc or grib
filename_example = f"{source}_{vname}_{time_beg}_{time_end}_E{exp}_{time_IC}.{filefmt}"
filename_example

'CWRF_AT2M_2010-09-01-06_2011-10-01-12_E01_2010-06-01-06.nc'

In [ ]:

#cmd = f"bash ./src/CFSdownloader {data_dir} {time_beg} {time_end} {IC_time} {vname}" # the downloader is goint to be stored in the src folder
cmd = f"bash ./src/CFSdownloader {data_dir} {IC_time} {vname}" 
check_call(cmd)

1

#Section 2 <font color='red'>**Regrider**</font>#

This section outlines the regridding process used for remapping data from the original grid of CFS (1deg x 1deg) to the CWRF grid cell.

To accomplish this task, a weighting file will be provided by Guangwei, which should be placed in the project's data folder. The NCL or CDO script/cmd will then be able to read this file and generate an output projected on the CWRF grid.

It is important to note that the projected files must follow the same filename convention as mentioned above.



In [ ]:
# here is the example command, please feel free to adjust it so we ca discuss on 
# the most proper choice
# I am not sure your implementation, potentially, you might want to do this by single date
# the point is that we can later on implement a simple parallel and just shell out the
# following command

filename_CFSraw= "CFSraw_AT2M_2010-09-01-06_2011-10-01-12_E01_2010-06-01-06.nc"
filename_CFS= "CFS_AT2M_2010-09-01-06_2011-10-01-12_E01_2010-06-01-06.nc"

cmd = f"ncl regrider.ncl {data_dir}/{filename_CFSraw}  {data_dir}/{filename_CFS}"

#Section 3 <font color='red'>**Transformer**</font>#


This section describes the process of transforming the time-frequency of the raw CFS data (which may also include CWRF data). The goal is to calculate the monthly/seasonal mean from daily values, and also calculate the corresponding anomaly.

The anomaly is a highly useful metric and is often the primary data used in subsequent analysis. Therefore, it is recommended to save these files with the prefix "a". For example, the CFS anomaly data should be labeled as follows: "a_CFS".


```
"aCFS_AT2M_2010-09-01-06_2011-10-01-12_E01_2010-06-01-06.nc"
```






In [ ]:
# I am inclined to use cdo, but if some specific functions are not 
#readily avialbe maybe we shall consider ncl for them
cmd = f"cdo -L monmean {data_dir}/{filename_CFSraw}  {data_dir}/monthly/{filename_CFS}"